In [1]:
import pandas as pd
import random
import itertools
import numpy as np

In [2]:
import itertools
import json
with open('category_mappings.json', 'r') as f:
    category_map = json.load(f)
    
id2category = {x[1]: x[0] for x in category_map.items()}
id2category

{0: '<UNK>',
 1: 'Сувениры',
 2: 'Фаст Фуд',
 3: 'Супермаркеты',
 4: 'Дом/Ремонт',
 5: 'Сервисные услуги',
 6: 'Красота',
 7: 'Разные товары',
 8: 'Транспорт',
 9: 'Медицинские услуги',
 10: 'Топливо',
 11: 'Одежда/Обувь',
 12: 'Наличные',
 13: 'Связь/Телеком',
 14: 'Частные услуги',
 15: 'Финансовые услуги',
 16: 'Рестораны',
 17: 'Развлечения',
 18: 'НКО',
 19: 'Книги',
 20: 'Кино',
 21: 'Автоуслуги',
 22: 'Музыка',
 23: 'Отели',
 24: 'Аптеки',
 25: 'Цветы',
 26: 'Ж/д билеты',
 27: 'Авиабилеты',
 28: 'Спорттовары',
 29: 'Госсборы',
 30: 'Аренда авто',
 31: 'Животные',
 32: 'Duty Free',
 33: 'Турагентства',
 34: 'Образование',
 35: 'Искусство',
 36: 'Фото/Видео'}

In [3]:
topics = {'Responsible consumption' : [11], 
          'Healthy food' : [2, 3], 
          'Waste reduction' : [10], 
          'Urbanism' : [10], 
          'Recycling' : [], 
          'Social Life' : []}

cater = {'Одежда/Обувь': 11,
        'Фаст Фуд': 2,
        'Супермаркеты': 3,
        'Топливо': 10}

In [4]:
advice = {}

advice[11] = ['Привет! Если Вы планируете в ближайшее время совершить покупку одежды или обуви, предлагаем Вам сдать ненужные вещи в пункты приёма либо в детские дома.', 
             'У H&M есть программа, по которой вы получаете скидку в 15% на покупку новых вещей при сдаче пакета с неиспользуемой одеждой.',
             'Старайтесь покупать вещи из натурального хлопка, так как при его производстве не выделяются вредные вещества и согласно сертификациями сотрудники обеспечены достойными условиями труда.',
             'Не покупайте вещи, содержащие синтетические материалы (микроволокно и тп). Данные вещества долго разлагаются и загрязняют мировой океан.']

advice[10] = ['Пересаживайтесь на общественный транспорт, это и дешевле, и часто занимает меньше времени.', 
             'Пересаживаясь на общественный транспорт, вы снижаете нагрузку на городскую инфраструктуру, уменьшаете выбросы в атмосферу и даёте людям возможность свободно дышать.',
             'Виды экологичного топлива: газомоторное топливо, электричество, биотопливо.']

advice[2] = ['Не покупайте много товаров с повышенным содержанием сахара.',
             'Рассмотрите возможность замены любимого фаст фуд ресторана на ресторан/кафе с более здоровым меню.',
             'Попкорн намного полезнее чипсов.']

advice[3] = ['Не покупайте много товаров с повышенным содержанием сахара.',
             'Добавляйте в свой рацион больше фруктов и овощей.',
             'Не совершайте покупки без списка, чтобы не брать товары импульсивно.',
             'Не ходите в магазин голодным.',
             'Увеличьте потребление протеина. Одно из исследований показало, что увеличение его в рационе на 30% помогло людям есть на 450 ккал в день меньше.',
             'Используйте более здоровые типы масел: оливковое, подсолнечное, кунжутное, кедровое.',
             'Ходите в магазин со своим пакетом или покупайте биоразлагаемые.']

In [5]:
file = 'category_scores.csv'

predictions = pd.read_csv(file, sep=';')

predictions.recommendations = predictions.recommendations.apply(lambda x: np.array(list(map(float, x.split(',')))))
predictions.head()

,party_rk,recommendations
0,20337,"[0.0274, 0.7683, 0.9116, 0.4016, 0.1475, 0.307..."
1,63404,"[0.0102, 0.2903, 0.6858, 0.0906, 0.0156, 0.093..."
2,24789,"[0.0638, 0.7253, 0.9646, 0.9187, 0.4675, 0.168..."
3,57970,"[0.1499, 0.9864, 0.9887, 0.7584, 0.5017, 0.489..."
4,12232,"[0.1031, 0.9425, 0.9875, 0.7744, 0.4344, 0.537..."


In [6]:
def get_probability_by_rk(rk):
    index = predictions[predictions['party_rk']==rk].index[0]
    party_rk, recommendations = predictions.iloc[index]
    return dict(zip(list(id2category.values())[1:], recommendations))

def get_predictions_by_rk(rk):
    index = predictions[predictions['party_rk']==rk].index[0]
    party_rk, recommendations = predictions.iloc[index]
    sorted_ids = (-recommendations).argsort() + 1
    return sorted_ids

def get_advice_by_idx(idx , interests):
    l = [topics[interes] for interes in interests]
    possible_idx = [val for sublist in l for val in sublist]
    if idx in possible_idx:
        return random.choice(advice[idx])
    else:
        return 'Советов нет'
    
def get_advice_user(user, int_1, int_2):
    dict_of_prob = get_probability_by_rk(user)
    print(dict_of_prob)
    best_categories = get_predictions_by_rk(user)[0]
    print('best category : {}\n'.format(best_categories))
    print("User's interests: {} and {}\n".format(list(topics.keys())[int_1], list(topics.keys())[int_2]) )
    print('Our advice is:', get_advice_by_idx(get_predictions_by_rk(user)[0], [list(topics.keys())[int_1], list(topics.keys())[int_2]]))
    return get_advice_by_idx(get_predictions_by_rk(user)[0], [list(topics.keys())[int_1], list(topics.keys())[int_2]])
    
    

In [7]:
person1 = predictions['party_rk'][0]
our_advice = get_advice_user(person1, 0, 1)

{'Сувениры': 0.0274, 'Фаст Фуд': 0.7683, 'Супермаркеты': 0.9116, 'Дом/Ремонт': 0.4016, 'Сервисные услуги': 0.1475, 'Красота': 0.3074, 'Разные товары': 0.569, 'Транспорт': 0.3425, 'Медицинские услуги': 0.1008, 'Топливо': 0.9074, 'Одежда/Обувь': 0.5104, 'Наличные': 0.3545, 'Связь/Телеком': 0.2097, 'Частные услуги': 0.0451, 'Финансовые услуги': 0.8696, 'Рестораны': 0.363, 'Развлечения': 0.132, 'НКО': 0.1202, 'Книги': 0.1203, 'Кино': 0.1316, 'Автоуслуги': 0.2297, 'Музыка': 0.217, 'Отели': 0.0409, 'Аптеки': 0.5377, 'Цветы': 0.0568, 'Ж/д билеты': 0.0303, 'Авиабилеты': 0.0319, 'Спорттовары': 0.0708, 'Госсборы': 0.1389, 'Аренда авто': 0.0189, 'Животные': 0.2746, 'Duty Free': 0.0084, 'Турагентства': 0.0064, 'Образование': 0.0347, 'Искусство': 0.0029, 'Фото/Видео': 0.0194}
best category : 3

User's interests: Responsible consumption and Healthy food

Our advice is: Ходите в магазин со своим пакетом или покупайте биоразлагаемые.


In [8]:
person2 = predictions['party_rk'][28]
our_advice = get_advice_user(person2, 4, 2)

{'Сувениры': 0.0274, 'Фаст Фуд': 0.9444, 'Супермаркеты': 0.9612, 'Дом/Ремонт': 0.7319, 'Сервисные услуги': 0.1134, 'Красота': 0.1168, 'Разные товары': 0.6609, 'Транспорт': 0.5207, 'Медицинские услуги': 0.0724, 'Топливо': 0.9646, 'Одежда/Обувь': 0.3478, 'Наличные': 0.5451, 'Связь/Телеком': 0.3087, 'Частные услуги': 0.0281, 'Финансовые услуги': 0.8843, 'Рестораны': 0.4602, 'Развлечения': 0.1506, 'НКО': 0.1136, 'Книги': 0.0611, 'Кино': 0.0982, 'Автоуслуги': 0.6348, 'Музыка': 0.0195, 'Отели': 0.0495, 'Аптеки': 0.5835, 'Цветы': 0.0941, 'Ж/д билеты': 0.0289, 'Авиабилеты': 0.0325, 'Спорттовары': 0.1068, 'Госсборы': 0.1813, 'Аренда авто': 0.0192, 'Животные': 0.063, 'Duty Free': 0.0037, 'Турагентства': 0.0058, 'Образование': 0.0107, 'Искусство': 0.0015, 'Фото/Видео': 0.0188}
best category : 10

User's interests: Recycling and Waste reduction

Our advice is: Пересаживаясь на общественный транспорт, вы снижаете нагрузку на городскую инфраструктуру, уменьшаете выбросы в атмосферу и даёте людям воз

In [9]:
person3 = predictions['party_rk'][3581]
our_advice = get_advice_user(person3, 0, 4)

{'Сувениры': 0.083, 'Фаст Фуд': 0.6679, 'Супермаркеты': 0.6655, 'Дом/Ремонт': 0.371, 'Сервисные услуги': 0.2115, 'Красота': 0.7323, 'Разные товары': 0.5426, 'Транспорт': 0.1563, 'Медицинские услуги': 0.4935, 'Топливо': 0.4192, 'Одежда/Обувь': 0.8743, 'Наличные': 0.5175, 'Связь/Телеком': 0.1737, 'Частные услуги': 0.1433, 'Финансовые услуги': 0.6803, 'Рестораны': 0.5177, 'Развлечения': 0.2405, 'НКО': 0.0864, 'Книги': 0.1229, 'Кино': 0.0544, 'Автоуслуги': 0.0611, 'Музыка': 0.0078, 'Отели': 0.0708, 'Аптеки': 0.571, 'Цветы': 0.0918, 'Ж/д билеты': 0.0254, 'Авиабилеты': 0.0764, 'Спорттовары': 0.1091, 'Госсборы': 0.0675, 'Аренда авто': 0.0123, 'Животные': 0.0462, 'Duty Free': 0.0265, 'Турагентства': 0.0147, 'Образование': 0.2053, 'Искусство': 0.0033, 'Фото/Видео': 0.0215}
best category : 11

User's interests: Responsible consumption and Recycling

Our advice is: Привет! Если Вы планируете в ближайшее время совершить покупку одежды или обуви, предлагаем Вам сдать ненужные вещи в пункты приёма л